# Smart Absensi Berbasis Wajah

Sistem absensi realtime menggunakan face recognition dengan Python.

---

## Fitur Utama:
1. **Registrasi Wajah** - Mendaftarkan wajah baru ke sistem
2. **Absensi Realtime** - Deteksi dan pencatatan absensi otomatis
3. **Visualisasi Data** - Melihat rekap absensi

---


## A. KONFIGURASI AWAL

### Import Library dan Cek Versi


In [1]:
# Import semua library yang diperlukan
import cv2
import face_recognition
import numpy as np
import pandas as pd
import pickle
import os
from datetime import datetime
import time

# Cek versi library
print("=" * 50)
print("VERSI LIBRARY")
print("=" * 50)
print(f"OpenCV: {cv2.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"Face Recognition: {face_recognition.__version__}")
print("=" * 50)
print("✓ Semua library berhasil diimport!")


C:\Users\DELL\miniconda3\envs\absensi-wajah\lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


VERSI LIBRARY
OpenCV: 4.11.0
NumPy: 1.26.4
Pandas: 2.3.1
Face Recognition: 1.2.3
✓ Semua library berhasil diimport!


### Inisialisasi Folder dan File


In [2]:
# Buat folder jika belum ada
folders = ['dataset_wajah', 'encodings', 'output']
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"✓ Folder '{folder}' dibuat")
    else:
        print(f"✓ Folder '{folder}' sudah ada")

# Buat file absensi.csv jika belum ada
if not os.path.exists('absensi.csv'):
    df = pd.DataFrame(columns=['nama', 'id', 'waktu'])
    df.to_csv('absensi.csv', index=False)
    print("✓ File 'absensi.csv' dibuat")
else:
    print("✓ File 'absensi.csv' sudah ada")

print("\n✓ Inisialisasi selesai!")


✓ Folder 'dataset_wajah' sudah ada
✓ Folder 'encodings' sudah ada
✓ Folder 'output' sudah ada
✓ File 'absensi.csv' sudah ada

✓ Inisialisasi selesai!


### Test Kamera

Fungsi untuk mencari kamera yang tersedia dan menguji apakah kamera berfungsi.


In [3]:
def find_camera():
    """
    Mencari kamera yang tersedia dengan mencoba index 0, 1, dan 2
    Returns: camera index yang berfungsi atau None
    """
    for i in range(3):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            print(f"✓ Kamera ditemukan di index {i}")
            return i, cap
        cap.release()
    return None, None

# Test kamera
print("Mencari kamera yang tersedia...")
camera_index, test_cap = find_camera()

if test_cap is None:
    print("\n❌ ERROR: Tidak ada kamera yang terdeteksi!")
    print("Solusi:")
    print("1. Pastikan kamera laptop berfungsi")
    print("2. Pastikan tidak ada aplikasi lain yang menggunakan kamera")
    print("3. Coba restart kernel dan jalankan ulang")
else:
    print("\nTekan 'q' untuk keluar dari preview kamera...\n")
    print("Membuka preview kamera dalam 2 detik...")
    time.sleep(2)
    
    # Tampilkan preview kamera selama 5 detik atau sampai 'q' ditekan
    start_time = time.time()
    frame_count = 0
    
    while True:
        ret, frame = test_cap.read()
        if not ret:
            print("❌ Gagal membaca frame dari kamera")
            break
        
        frame_count += 1
        
        # Tambahkan teks ke frame
        cv2.putText(frame, "TEST KAMERA - Tekan 'q' untuk keluar", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Frame: {frame_count}", 
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        cv2.imshow('Test Kamera', frame)
        
        # Keluar jika 'q' ditekan atau sudah 10 detik
        if cv2.waitKey(1) & 0xFF == ord('q') or (time.time() - start_time) > 10:
            break
    
    # Tutup kamera dengan benar
    test_cap.release()
    cv2.destroyAllWindows()
    
    # Tunggu sebentar untuk memastikan window tertutup
    for i in range(4):
        cv2.waitKey(1)
    
    print(f"\n✓ Test kamera selesai! ({frame_count} frame berhasil dibaca)")
    print(f"✓ Kamera berfungsi dengan baik di index {camera_index}")


Mencari kamera yang tersedia...
✓ Kamera ditemukan di index 0

Tekan 'q' untuk keluar dari preview kamera...

Membuka preview kamera dalam 2 detik...

✓ Test kamera selesai! (231 frame berhasil dibaca)
✓ Kamera berfungsi dengan baik di index 0


---

## B. MODUL REGISTRASI WAJAH

### Fungsi Registrasi Wajah Baru


In [14]:
def registrasi_wajah(nama, id_mahasiswa):
    """
    Registrasi wajah baru ke sistem
    
    Parameters:
    - nama: Nama lengkap (string)
    - id_mahasiswa: NIM atau ID unik (string)
    
    Returns:
    - True jika berhasil, False jika gagal
    """
    
    print("=" * 60)
    print("REGISTRASI WAJAH")
    print("=" * 60)
    print(f"Nama: {nama}")
    print(f"ID: {id_mahasiswa}")
    print("\nInstruksi:")
    print("1. Pastikan wajah Anda terlihat jelas di kamera")
    print("2. Tekan 'c' untuk capture foto")
    print("3. Tekan 'q' untuk keluar\n")
    
    # Cari kamera
    camera_index, cap = find_camera()
    if cap is None:
        print("❌ Kamera tidak dapat diakses")
        return False
    
    wajah_terdeteksi = False
    foto_tersimpan = False
    
    print("Membuka kamera...\n")
    time.sleep(1)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Gagal membaca frame")
            break
        
        # Deteksi wajah
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        
        # Gambar kotak di sekitar wajah
        display_frame = frame.copy()
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(display_frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(display_frame, "Wajah Terdeteksi", (left, top - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # Tambahkan informasi
        if len(face_locations) > 0:
            status_text = "STATUS: Wajah terdeteksi - Tekan 'c' untuk capture"
            color = (0, 255, 0)
            wajah_terdeteksi = True
        else:
            status_text = "STATUS: Posisikan wajah di depan kamera"
            color = (0, 0, 255)
            wajah_terdeteksi = False
        
        cv2.putText(display_frame, status_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(display_frame, f"Nama: {nama} | ID: {id_mahasiswa}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(display_frame, "'c' = capture | 'q' = keluar", (10, display_frame.shape[0] - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Registrasi Wajah', display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        # Capture foto
        if key == ord('c'):
            if wajah_terdeteksi and len(face_locations) > 0:
                # Simpan foto
                filename = f"{nama}_{id_mahasiswa}.jpg"
                filepath = os.path.join('dataset_wajah', filename)
                cv2.imwrite(filepath, frame)
                print(f"\n✓ Foto berhasil disimpan: {filepath}")
                
                # Encode wajah
                print("Membuat encoding wajah...")
                face_encoding = face_recognition.face_encodings(rgb_frame, face_locations)[0]
                
                # Simpan encoding
                encoding_data = {
                    'nama': nama,
                    'id': id_mahasiswa,
                    'encoding': face_encoding
                }
                
                encoding_filename = f"{nama}_{id_mahasiswa}.pickle"
                encoding_filepath = os.path.join('encodings', encoding_filename)
                
                with open(encoding_filepath, 'wb') as f:
                    pickle.dump(encoding_data, f)
                
                print(f"✓ Encoding berhasil disimpan: {encoding_filepath}")
                print("\n" + "=" * 60)
                print("✓ REGISTRASI BERHASIL!")
                print("=" * 60)
                
                foto_tersimpan = True
                time.sleep(2)
                break
            else:
                print("\n⚠ Tidak ada wajah yang terdeteksi. Posisikan wajah dengan benar.")
        
        # Keluar
        elif key == ord('q'):
            print("\nRegistrasi dibatalkan.")
            break
    
    # Tutup kamera
    cap.release()
    cv2.destroyAllWindows()
    for i in range(4):
        cv2.waitKey(1)
    
    return foto_tersimpan

print("✓ Fungsi registrasi_wajah siap digunakan!")


✓ Fungsi registrasi_wajah siap digunakan!


### Contoh Penggunaan Registrasi

**Jalankan cell di bawah untuk mendaftarkan wajah baru:**


In [15]:
# CONTOH REGISTRASI - Edit nama dan ID sesuai kebutuhan

nama_mahasiswa = input("Masukkan Nama: ")
nim_mahasiswa = input("Masukkan NIM/ID: ")

# Validasi input
if nama_mahasiswa.strip() == "" or nim_mahasiswa.strip() == "":
    print("❌ Nama dan ID tidak boleh kosong!")
else:
    hasil = registrasi_wajah(nama_mahasiswa, nim_mahasiswa)
    if hasil:
        print("\n✓ Registrasi selesai! Wajah berhasil didaftarkan ke sistem.")
    else:
        print("\n❌ Registrasi gagal. Silakan coba lagi.")


Masukkan Nama:  Fahren
Masukkan NIM/ID:  23215030


REGISTRASI WAJAH
Nama: Fahren
ID: 23215030

Instruksi:
1. Pastikan wajah Anda terlihat jelas di kamera
2. Tekan 'c' untuk capture foto
3. Tekan 'q' untuk keluar

✓ Kamera ditemukan di index 0
Membuka kamera...


✓ Foto berhasil disimpan: dataset_wajah\Fahren_23215030.jpg
Membuat encoding wajah...
✓ Encoding berhasil disimpan: encodings\Fahren_23215030.pickle

✓ REGISTRASI BERHASIL!

✓ Registrasi selesai! Wajah berhasil didaftarkan ke sistem.


---

## C. MODUL ABSENSI WAJAH (REALTIME)

### Load Semua Encoding Wajah yang Terdaftar


In [6]:
def load_encodings():
    """
    Load semua encoding wajah dari folder encodings/
    
    Returns:
    - known_encodings: list of face encodings
    - known_names: list of dictionaries {'nama': ..., 'id': ...}
    """
    known_encodings = []
    known_data = []
    
    encoding_files = [f for f in os.listdir('encodings') if f.endswith('.pickle')]
    
    if len(encoding_files) == 0:
        print("⚠ Tidak ada encoding yang ditemukan!")
        print("Silakan registrasi wajah terlebih dahulu.")
        return [], []
    
    print(f"Loading {len(encoding_files)} encoding...")
    
    for filename in encoding_files:
        filepath = os.path.join('encodings', filename)
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            known_encodings.append(data['encoding'])
            known_data.append({
                'nama': data['nama'],
                'id': data['id']
            })
            print(f"  ✓ {data['nama']} ({data['id']})")
    
    print(f"\n✓ Total {len(known_encodings)} wajah berhasil di-load!")
    return known_encodings, known_data

print("✓ Fungsi load_encodings siap digunakan!")


✓ Fungsi load_encodings siap digunakan!


### Fungsi Absensi Realtime


In [16]:
def absensi_realtime():
    """
    Jalankan sistem absensi realtime dengan deteksi wajah
    """
    
    print("=" * 60)
    print("SISTEM ABSENSI REALTIME")
    print("=" * 60)
    
    # Load encoding
    known_encodings, known_data = load_encodings()
    
    if len(known_encodings) == 0:
        return
    
    # Cari kamera
    camera_index, cap = find_camera()
    if cap is None:
        print("❌ Kamera tidak dapat diakses")
        return
    
    # Set untuk tracking siapa yang sudah absen di sesi ini
    sudah_absen = set()
    
    print("\nInstruksi:")
    print("- Posisikan wajah di depan kamera")
    print("- Sistem akan otomatis mendeteksi dan mencatat absensi")
    print("- Tekan 'q' untuk keluar\n")
    print("Membuka kamera...\n")
    time.sleep(2)
    
    frame_count = 0
    process_every_n_frames = 2  # Process setiap 2 frame untuk performa
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Gagal membaca frame")
            break
        
        frame_count += 1
        
        # Process hanya setiap n frame
        if frame_count % process_every_n_frames == 0:
            # Resize untuk performa (0.25x)
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
            
            # Deteksi wajah
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
            
            # Loop untuk setiap wajah yang terdeteksi
            for face_encoding, face_location in zip(face_encodings, face_locations):
                # Bandingkan dengan database
                matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.6)
                face_distances = face_recognition.face_distance(known_encodings, face_encoding)
                
                best_match_index = np.argmin(face_distances)
                
                if matches[best_match_index]:
                    # Wajah cocok!
                    person_data = known_data[best_match_index]
                    nama = person_data['nama']
                    person_id = person_data['id']
                    confidence = (1 - face_distances[best_match_index]) * 100
                    
                    # Scale kembali lokasi wajah
                    top, right, bottom, left = face_location
                    top *= 4
                    right *= 4
                    bottom *= 4
                    left *= 4
                    
                    # Gambar kotak dan nama
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, f"{nama} ({person_id})", (left + 6, bottom - 6),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    
                    # Catat absensi jika belum absen
                    if person_id not in sudah_absen:
                        # Simpan ke CSV
                        waktu_sekarang = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        
                        df = pd.read_csv('absensi.csv')
                        new_row = pd.DataFrame([{
                            'nama': nama,
                            'id': person_id,
                            'waktu': waktu_sekarang
                        }])
                        df = pd.concat([df, new_row], ignore_index=True)
                        df.to_csv('absensi.csv', index=False)
                        
                        # Simpan foto absensi
                        output_filename = f"{nama}_{person_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
                        output_path = os.path.join('output', output_filename)
                        cv2.imwrite(output_path, frame)
                        
                        # Tandai sudah absen
                        sudah_absen.add(person_id)
                        
                        print(f"\n{'='*60}")
                        print(f"✓ ABSENSI TERCATAT")
                        print(f"{'='*60}")
                        print(f"Nama       : {nama}")
                        print(f"ID         : {person_id}")
                        print(f"Waktu      : {waktu_sekarang}")
                        print(f"Confidence : {confidence:.2f}%")
                        print(f"Foto       : {output_path}")
                        print(f"{'='*60}\n")
                else:
                    # Wajah tidak dikenali
                    top, right, bottom, left = face_location
                    top *= 4
                    right *= 4
                    bottom *= 4
                    left *= 4
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    cv2.putText(frame, "Tidak Dikenali", (left + 6, bottom - 6),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Tambahkan info ke frame
        cv2.putText(frame, "SISTEM ABSENSI - Tekan 'q' untuk keluar", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Sudah Absen: {len(sudah_absen)} orang", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Absensi Realtime', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Tutup kamera
    cap.release()
    cv2.destroyAllWindows()
    for i in range(4):
        cv2.waitKey(1)
    
    print("\n" + "=" * 60)
    print("SESI ABSENSI SELESAI")
    print("=" * 60)
    print(f"Total yang hadir: {len(sudah_absen)} orang")
    if len(sudah_absen) > 0:
        print("\nDaftar yang hadir:")
        for person_id in sudah_absen:
            for data in known_data:
                if data['id'] == person_id:
                    print(f"  - {data['nama']} ({person_id})")
    print("=" * 60)

print("✓ Fungsi absensi_realtime siap digunakan!")


✓ Fungsi absensi_realtime siap digunakan!


In [17]:
# JALANKAN ABSENSI REALTIME
absensi_realtime()


SISTEM ABSENSI REALTIME
Loading 1 encoding...
  ✓ Fahren (23215030)

✓ Total 1 wajah berhasil di-load!
✓ Kamera ditemukan di index 0

Instruksi:
- Posisikan wajah di depan kamera
- Sistem akan otomatis mendeteksi dan mencatat absensi
- Tekan 'q' untuk keluar

Membuka kamera...


✓ ABSENSI TERCATAT
Nama       : Fahren
ID         : 23215030
Waktu      : 2025-11-28 21:27:43
Confidence : 61.44%
Foto       : output\Fahren_23215030_20251128_212743.jpg


SESI ABSENSI SELESAI
Total yang hadir: 1 orang

Daftar yang hadir:
  - Fahren (23215030)


---

## D. VISUALISASI DATA ABSENSI

### Lihat Data Absensi


In [9]:
# Load data absensi
df_absensi = pd.read_csv('absensi.csv')

if len(df_absensi) == 0:
    print("⚠ Belum ada data absensi.")
else:
    print("=" * 80)
    print("DATA ABSENSI")
    print("=" * 80)
    print(f"\nTotal Records: {len(df_absensi)}\n")
    display(df_absensi)


DATA ABSENSI

Total Records: 1



,nama,id,waktu
0,Fahren,23215030,2025-11-28 21:14:45


### Statistik Absensi


In [18]:
if len(df_absensi) > 0:
    print("=" * 80)
    print("STATISTIK ABSENSI")
    print("=" * 80)
    
    # Hitung jumlah absensi per orang
    absensi_per_orang = df_absensi.groupby(['nama', 'id']).size().reset_index(name='jumlah_absensi')
    absensi_per_orang = absensi_per_orang.sort_values('jumlah_absensi', ascending=False)
    
    print("\n1. Jumlah Absensi per Orang:")
    print("=" * 80)
    display(absensi_per_orang)
    
    # Absensi terbaru
    print("\n2. 10 Absensi Terbaru:")
    print("=" * 80)
    display(df_absensi.tail(10))
    
    # Ringkasan
    print("\n3. Ringkasan:")
    print("=" * 80)
    print(f"Total Absensi      : {len(df_absensi)} kali")
    print(f"Jumlah Orang Unik  : {df_absensi['id'].nunique()} orang")
    print(f"Absensi Pertama    : {df_absensi['waktu'].min() if len(df_absensi) > 0 else '-'}")
    print(f"Absensi Terakhir   : {df_absensi['waktu'].max() if len(df_absensi) > 0 else '-'}")
    print("=" * 80)
else:
    print("⚠ Belum ada data untuk ditampilkan.")


STATISTIK ABSENSI

1. Jumlah Absensi per Orang:


,nama,id,jumlah_absensi
0,Fahren,23215030,1



2. 10 Absensi Terbaru:


,nama,id,waktu
0,Fahren,23215030,2025-11-28 21:14:45



3. Ringkasan:
Total Absensi      : 1 kali
Jumlah Orang Unik  : 1 orang
Absensi Pertama    : 2025-11-28 21:14:45
Absensi Terakhir   : 2025-11-28 21:14:45


### Export Data ke Excel (Opsional)


In [19]:
# Export ke Excel jika ada data
if len(df_absensi) > 0:
    try:
        output_excel = f"laporan_absensi_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df_absensi.to_excel(output_excel, index=False)
        print(f"✓ Data berhasil di-export ke: {output_excel}")
    except:
        print("⚠ Gagal export ke Excel. Install openpyxl dengan: pip install openpyxl")
        # Export ke CSV sebagai alternatif
        output_csv = f"laporan_absensi_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_absensi.to_csv(output_csv, index=False)
        print(f"✓ Data berhasil di-export ke CSV: {output_csv}")
else:
    print("⚠ Tidak ada data untuk di-export.")


⚠ Gagal export ke Excel. Install openpyxl dengan: pip install openpyxl
✓ Data berhasil di-export ke CSV: laporan_absensi_20251128_212818.csv


---

## E. UTILITY FUNCTIONS

### Lihat Daftar Wajah Terdaftar


In [20]:
def lihat_wajah_terdaftar():
    """
    Menampilkan daftar semua wajah yang sudah terdaftar
    """
    encoding_files = [f for f in os.listdir('encodings') if f.endswith('.pickle')]
    
    print("=" * 60)
    print("DAFTAR WAJAH TERDAFTAR")
    print("=" * 60)
    
    if len(encoding_files) == 0:
        print("\n⚠ Belum ada wajah yang terdaftar.")
        print("Silakan registrasi wajah terlebih dahulu.\n")
        return
    
    print(f"\nTotal: {len(encoding_files)} wajah terdaftar\n")
    
    data_list = []
    for i, filename in enumerate(encoding_files, 1):
        filepath = os.path.join('encodings', filename)
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            data_list.append({
                'No': i,
                'Nama': data['nama'],
                'ID': data['id'],
                'File': filename
            })
    
    df_terdaftar = pd.DataFrame(data_list)
    display(df_terdaftar)
    print("\n" + "=" * 60)

# Jalankan fungsi
lihat_wajah_terdaftar()


DAFTAR WAJAH TERDAFTAR

Total: 1 wajah terdaftar



,No,Nama,ID,File
0,1,Fahren,23215030,Fahren_23215030.pickle


### Reset Data Absensi (Hati-hati!)


In [22]:
def reset_absensi():
    """
    Reset/hapus semua data absensi
    HATI-HATI: Ini akan menghapus semua record absensi!
    """
    konfirmasi = input("Apakah Anda yakin ingin menghapus semua data absensi? (ketik 'HAPUS' untuk konfirmasi): ")
    
    if konfirmasi == "HAPUS":
        # Backup data lama
        if os.path.exists('absensi.csv'):
            backup_name = f"absensi_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            os.rename('absensi.csv', backup_name)
            print(f"✓ Backup data lama disimpan: {backup_name}")
        
        # Buat file baru
        df = pd.DataFrame(columns=['nama', 'id', 'waktu'])
        df.to_csv('absensi.csv', index=False)
        print("✓ Data absensi berhasil direset!")
    else:
        print("❌ Reset dibatalkan.")

# Uncomment baris di bawah jika ingin reset
reset_absensi()


Apakah Anda yakin ingin menghapus semua data absensi? (ketik 'HAPUS' untuk konfirmasi):  HAPUS


✓ Backup data lama disimpan: absensi_backup_20251128_213410.csv
✓ Data absensi berhasil direset!


---

## SELESAI!

### Panduan Penggunaan:

1. **Registrasi Wajah Baru:**
   - Jalankan cell di bagian "B. MODUL REGISTRASI WAJAH"
   - Input nama dan ID
   - Posisikan wajah di depan kamera
   - Tekan 'c' untuk capture

2. **Mulai Absensi:**
   - Jalankan cell di bagian "C. MODUL ABSENSI WAJAH"
   - Sistem akan otomatis mendeteksi dan mencatat absensi
   - Tekan 'q' untuk keluar

3. **Lihat Data:**
   - Jalankan cell di bagian "D. VISUALISASI DATA ABSENSI"
   - Lihat tabel dan statistik absensi

### Tips:
- Pastikan pencahayaan cukup saat registrasi dan absensi
- Posisikan wajah menghadap langsung ke kamera
- Jangan terlalu jauh atau terlalu dekat dari kamera
- Sistem dapat mendeteksi beberapa wajah sekaligus

### Troubleshooting:
- **Kamera tidak berfungsi:** Periksa apakah kamera digunakan aplikasi lain
- **Wajah tidak terdeteksi:** Perbaiki pencahayaan dan posisi wajah
- **Salah mengenali wajah:** Registrasi ulang dengan foto lebih jelas

---

**Sistem Smart Absensi Berbasis Wajah siap digunakan!** 🎉
